In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [7]:

text = pd.read_csv("PRSA_data.csv",encoding='unicode_escape').convert_dtypes()
text.isnull().sum()
text

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,<NA>,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,<NA>,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,<NA>,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,<NA>,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,<NA>,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10,-22,-3.0,1034.0,NW,242.7,0,0
43822,43823,2014,12,31,22,8,-22,-4.0,1034.0,NW,246.72,0,0


In [8]:
dummies = pd.get_dummies(text['cbwd'])  #对Car Model字段用get_dummies数字化
merged = pd.concat([text,dummies],axis='columns')
text = merged.drop(['cbwd'],axis='columns')
dummies

,NE,NW,SE,cv
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
43819,0,1,0,0
43820,0,1,0,0
43821,0,1,0,0
43822,0,1,0,0


In [5]:

def generate_dataset(data):
#先划分数据集
    Train_set=pd.DataFrame(columns=data.columns)
    Test_set=pd.DataFrame(columns=data.columns)
    text_data =data.groupby(['year','month','day'])
    for index,(name,data)  in enumerate(text_data):
        if((index+1)%7 ==0):
            Test_set =Test_set.append(data)
        else:
            Train_set = Train_set.append(data)

    return Train_set,Test_set
Train_set,Test_set =generate_dataset(text)

(          No  year month day hour pm2.5 DEWP  TEMP    PRES     Iws Is Ir NE  \
 0          1  2010     1   1    0  <NA>  -21 -11.0  1021.0    1.79  0  0  0   
 1          2  2010     1   1    1  <NA>  -21 -12.0  1020.0    4.92  0  0  0   
 2          3  2010     1   1    2  <NA>  -21 -11.0  1019.0    6.71  0  0  0   
 3          4  2010     1   1    3  <NA>  -21 -14.0  1019.0    9.84  0  0  0   
 4          5  2010     1   1    4  <NA>  -20 -12.0  1018.0   12.97  0  0  0   
 ...      ...   ...   ...  ..  ...   ...  ...   ...     ...     ... .. .. ..   
 43819  43820  2014    12  31   19     8  -23  -2.0  1034.0  231.97  0  0  0   
 43820  43821  2014    12  31   20    10  -22  -3.0  1034.0  237.78  0  0  0   
 43821  43822  2014    12  31   21    10  -22  -3.0  1034.0   242.7  0  0  0   
 43822  43823  2014    12  31   22     8  -22  -4.0  1034.0  246.72  0  0  0   
 43823  43824  2014    12  31   23    12  -21  -3.0  1034.0  249.85  0  0  0   
 
       NW SE cv  
 0      1  0  0  
 1

In [ ]:
#缺失值进行处理：
Train_set  = Train_set.dropna()
Test_set= Test_set.dropna()
Train_set = Train_set.astype('int')
Test_set =Test_set.astype('int')

In [5]:
#这个可能会有异常值
text['DEWP'].value_counts()
text['TEMP'].value_counts()
text['PRES'].value_counts()



In [8]:

Y_train = Train_set['pm2.5']
X_train = Train_set.drop(['pm2.5'],axis='columns')

Y_test = Test_set['pm2.5']
X_test = Test_set.drop(['pm2.5'],axis='columns')


In [9]:
#普通的线性回归
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,Y_train)  #用LinearRegression拟合训练数据
model.score(X_test,Y_test)  #计算得分


0.2954569659782572

In [10]:
from sklearn.linear_model import RidgeCV
alphas = np.logspace(-10,10,200)
lr_cv = RidgeCV(alphas=alphas).fit(X_train,Y_train)
lr_cv.alpha_

0.0017225859653987874

In [11]:
from sklearn.linear_model import Ridge
lr = Ridge()
# lr = Ridge(alpha=0.0017225859653987874)
lr.fit(X_train,Y_train)
lr.score(X_test,Y_test)


0.296672964938143